<a href="https://colab.research.google.com/github/valentinagliozzi/NNCourse/blob/main/LabCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Convolutional Neural Networks** (CNNs)



Le [reti convoluzionali](https://en.wikipedia.org/wiki/Convolutional_neural_network) (CNN) sono un tipo di rete neurale artificiale progettata per l'elaborazione efficiente di dati in forma matriciale (o tensoriale), tra cui le immagini sono l'esempio più comune.

Queste reti hanno rivoluzionato il campo dell'elaborazione di immagini, e hanno sancito il diffonfersi del deep learning. In molti problemi, tra cui analisi di immagini, riconoscimento di pattern e visione artificiale, il deep learning costituisce lo stato dell'arte da circa 10 anni.

Le CNN utilizzano layer convoluzionali (cioè layer che effettuano la [convoluzione discreta](https://en.wikipedia.org/wiki/Convolution#Discrete_convolution)) per l'estrazione di features e layer fully connected, insieme per la risoluzione del task in esame (nel nostro caso, *classificazione multi-classe*).

![Immagine](https://mriquestions.com/uploads/3/4/5/7/34572113/cnn-sample-layout_orig.png)

Le CNNs sono complesse rispetto ai Multilayer Perceptron, perciò può non essere banale visualizzarne la struttura. Tuttavia, sul web sono state create delle ottime visualizzazioni, come "[An Interactive Node-Link Visualization of Convolutional Neural Networks](https://adamharley.com/nn_vis/)" di Adam W. Harley.

![Screenshot interactive node visualization](https://adamharley.com/nn_vis/images/convnet_flat_480.png)

---

# 🧪⚗️ <font color='lime'><u>**Laboratorio: classificazione di FashionMNIST con CNN**</u></font>

Utilizziamo [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist), un dataset contenente immagini a bassa risoluzione di capi di abbigliamento, fornito da Zalando per la classificazione. E' stato pensato come benchmark più difficile di [MNIST](https://en.wikipedia.org/wiki/MNIST_database), noto dataset contenente un ampio insieme di immagini di cifre scritte a mano.

Come framework per il Deep Learning utilizzeremo **PyTorch**.

### <font color='gold'>**Installazione dipendenze e importazione librerie**</font>

Installiamo le dipendenze e importiamo le librerie necessarie.

In [ ]:
%%capture
!pip install umap-learn torchviz torchview

In [ ]:
%%capture

# per visualizzazione dati
from sklearn.decomposition import PCA
from umap import UMAP

# per visualizzare grafici
import matplotlib.pyplot as plt

# componenti PyTorch
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

# varie
import numpy as np
from tqdm import tqdm
from types import SimpleNamespace
from torchviz import make_dot
from torchview import draw_graph
from sklearn.metrics import ConfusionMatrixDisplay

### <font color='gold'>**Creazione del dataset**</font>

PyTorch fornisce la classe `FashionMNIST` (`torchvision.datasets`) che racchiude il dataset sotto forma di lista di [PIL images](https://pillow.readthedocs.io/en/latest/reference/Image.html). Per convertirle in tensori, è sufficiente specificare l'uso della trasformazione `torchvision.ToTensor`.

In [ ]:
dataset = FashionMNIST(
    root='data',  # directory per download
    download=True,
    transform=ToTensor()  # trasformazione: PIL.image ==> torch.Tensor
    )
target_names =  ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\nInformazioni sul dataset\n' + '-'*100)
print(f'Numero di esempi: {len(dataset):,}')
print('Numero di classi:', len(set(dataset.targets.tolist())))

### <font color='gold'>**Partizionamento del dataset: train, validation, test**</font>

Usando la funzione `torch.utils.data.random_split` dividiamo randomicamente il dataset in 3 porzioni:

- <u>Training set</u> (80%): contiene i dati che saranno usati per trainare la rete
- <u>Test set</u> (10%): contiene i dati che saranno usato **soltanto alla fine**, per testare le performance della rete
- <u>Validation set</u> (10%): contiene i dati che saranno usati periodicamente durante il training per stimare la performance che la rete ha sul test set.

➡️ **Nota bene**: i dati presenti nel validation set non sono usati per trainare la rete

In [ ]:
seed_random_split = 2023

random_generator = torch.Generator().manual_seed(seed_random_split)
train_dataset, val_dataset, test_dataset = random_split(dataset, lengths=[0.8, 0.1, 0.1], generator=random_generator)

print('Random split dataset\n' + '-'*100)
print(f'Training set:\t{len(train_dataset):,}')
print(f'Validation set:\t{len(val_dataset):,}')
print(f'Test set:\t{len(test_dataset):,}')

###   <font color='gold'>**Osserviamo qualche informazione di un esempio del training set**</font>

Possiamo vedere che un esempio è un tensore a 3 dimensioni, riferite rispettivamente a:

1. numero di canali:
  - immagini RGB avranno 3 canali
  - immagini RGBA (A = alpha, opacità) avranno 4 canali
  - immagini grayscale (<u>il nostro caso</u>) 1 solo canale
2. altezza dell'immagine
3. larghezza dell'immagine

In [ ]:
print('Esempio numero 1\n' + '-'*100)
x, y = train_dataset[0]
print('Input:', x.shape, '\t ==> (channel, height, width)')
print('Classe:', y, f'({target_names[y]})')

### <font color='gold'>**Visualizziamo alcuni esempi (immagini)**</font>

Possiamo usare la funzione `matplotlib.pyplot.imshow` per visualizzare i tensori sotto forma di immagine. La funzione mappa ogni valore (normalizzato sul range [0,1]) ad un colore. In questo caso invertiamo il valore per avere delle immagini nere su sfondo bianco.

In [ ]:
plt.subplots(4,4, figsize=(10,10))
for i in range(16):
  x, y = train_dataset[i]
  plt.subplot(4, 4, i+1)
  plt.imshow(1-x[0], cmap='gray')
  plt.title(target_names[y])
  plt.axis('off')

### <font color='gold'>**Visualizziamo la distribuzione degli esempi (w/ dimensionality reduction)**</font>

Quando si deve risolvere un task è buona norma prima osservare attentamente il dataset.

Possiamo visualizzare quanto sono separate le classi proiettando i tensori degli esempi in uno spazio bidimensionale e visualizzandoli in uno *scatter plot*.

Possiamo ridurre i tensori corrispondenti agli esempi in coppie di numeri reali usando un approccio di *dimensionality reduction*.

Definiamo una *utility function* che ci permetta di visualizzare il dataset usando un dato metodo di dimensionality reduction (che sia conforme con l'interfaccia di `Scikit-Learn`)

In [ ]:
def plot_dataset_visualization(X, y, method, title):
  X_transformed = method.fit_transform(X)

  plt.figure(figsize=(8,8))
  for i, l in enumerate(target_names):
    X_plot = X_transformed[y == i]
    plt.scatter(X_plot[:,0], X_plot[:,1], label=l, c=f'C{i}', s=1)

  plt.legend(markerscale=8, loc=(1.01,0), frameon=False)
  plt.title(title)

Visualizziamo il dataset, colorando gli esempi della stessa classe con lo stesso colore, usando 2 metodi diversi di visualizzazione:

- [Principal Component Analisys](https://it.wikipedia.org/wiki/Analisi_delle_componenti_principali) (PCA). Con la PCA si effettua una trasformazione lineare del dataset in uno spazio di nuove features non correlate tra loro. Successivamente, si mantengono le *n* (2 nel nostro caso) features che catturano la massima varianza.

- [Uniform manifold approximation and projection](https://pair-code.github.io/understanding-umap/) (UMAP): Metodo di riduzione della dimensionalità non lineare che produce un risultato simile alla PCA, ma permette di visualizzare in maniera molto chiara la separazione tra classi.

In [ ]:
X = dataset.data
y = dataset.targets
X = X.reshape(len(X), -1)


visualizers = {
    'Principal Component Analysis': PCA(n_components=2),
    'Uniform Manifold Approximation and Projection': UMAP(n_components=2, n_epochs=50)
}

for name, method in visualizers.items():
  plot_dataset_visualization(X, y, method, name)
  plt.show()
  print('\n')

Possiamo osservare che le classi sono parzialmente sovrapposte, rendendo il problema di classificazion non banale.

### <font color='gold'>**Definizione della rete convoluzionale**</font>

In `PyTorch` per definire una rete, è necessario:

- estendere la classe `torch.nn.Module`
- definire i layer (o eventualmente i singoli parametri) della rete nella funzione `__init__`
- implementare la funzione `forward` che costituisce la passata in avanti della rete

Definiamo la rete convoluzionale usando layer offerti da PyTorch:

- `Conv2d` ci permette di definire convoluzioni, specificando canali di input/output, dimensione del kernel, stride, padding
- funzione di attivazione `ReLU`
- `MaxPool2d` specificando dimensione del kernel, stride, padding
- `BatchNorm2d` *normalizza* e *centra* l'input del layer. Stabilizza e accelera il training
- `Dropout` azzera alcune features dell'input del layer con probabilità $p$ (soltanto durante il training). Migliora la generalizzazione della rete.

Nella parte restante della rete utilizziamo anche:

- `Linear`, che definisce un singolo layer fully connected
- come livello finale `Softmax`, che calcola la probabilità di appartenenza alla $i$-esima classe come segue:

$$\text{softmax}(\mathbf{x})_i = \frac{\exp\left(x_i\right)}{\exp\left(\sum_{j=1}^d x_j\right)}$$

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding='valid'),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        nn.BatchNorm2d(16),
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding='valid'),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
        nn.BatchNorm2d(32)
    )
    self.fc = nn.Sequential(
        nn.Dropout(0.6), # 0.6
        nn.Linear(in_features=800, out_features=256),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(in_features=256, out_features=64),
        nn.ReLU(),
        nn.Dropout(0.2)
    )
    self.out = nn.Sequential(
        nn.Linear(in_features=64, out_features=len(target_names)),
        nn.Softmax(-1)
    )
    self.embeddings = None

  def forward(self, x):
    x = self.conv(x)

    # appiattisci dalla dimensione 1 in poi
    x = x.flatten(1)
    x = self.fc(x)

    # salviamo la rappresentazione vettoriale per la visualizzazione
    self.embeddings = x
    x = self.out(x)
    return x

print('Architettura della rete convoluzionale\n' + '-' * 100)
model = CNN()
print(model)

### 💡<font color='lightblue'>**Approfondimento: visualizzazione con GraphViz del modello e del grafo computazionale**</font>

Esistono delle librerie per visualizzare l'architettura sotto forma di grafo. Ad esempio la funzione `torchview.draw_graph`.

In [ ]:
x = torch.zeros(10, 1, 28, 28)
y = torch.zeros(10)
dot = draw_graph(model, input_data=x, expand_nested=True, graph_name='CNN', device='cpu')
dot.resize_graph(1.5)
dot.visual_graph

Altre librerie, come `torchviz`, ci permettono di visualizzare l'intero grafo computazionale. Ad esempio, calcoliamo l'output del modello e visualizziamo il grafo con la funzione `torchviz.make_dot`.

In [ ]:
pred = model(x)
dot = make_dot(pred, params=dict(list(model.named_parameters()) + [('Prediction', pred)]))
dot.node_attr.update({'width': '0.1', 'margin': '0.01'})
dot.graph_attr.update({'size': '50,50', 'nodesep': '0.2', 'ranksep': '0.25'})
dot

### <font color='gold'>**Funzione `evaluate`**</font>

Definiamo una *utility function* per valutare le performance di un modello su un dataset.
La funzione restituisce:

- <u>loss media</u> sul dataset
- <u>accuratezza</u> della predizione (assumendo che si selezioni la classe con probabilità massima)
- <u>info</u>: dizionario che contiene dei dati che ci serviranno per studiare il training
    - embedding (rappresentazione vettoriale prima del livello di output) degli esempi
    - distribuzione di probabilità restituita dal modello
    - predizione del modello (classe più probabile)
    - targets (true labels)

In [ ]:
def evaluate(model, dataset, args, no_loading_bar=True):
  dataloader = DataLoader(dataset, batch_size=args.batch_size, num_workers=2)

  tot_loss = 0
  n_correct = 0
  embeddings, preds, target, probs = [], [], [], []

  model.eval()

  with torch.no_grad():
    for (X, y) in tqdm(dataloader, disable=no_loading_bar):
      X, y = X.to(args.device), y.to(args.device)

      # calcolo delle probabilità di appartenenza alle classi
      prob = model(X)

      # calcolo delle predizioni: classe con probabilità massima
      pred = prob.argmax(-1)

      # calcolo della loss media sul batch (può esserre Mean-Squared Error, Cross Entropy, o altro...)
      loss = args.loss_fn(prob, y)

      n_correct += sum(pred == y)
      tot_loss += loss.item() * X.shape[0]

      embeddings.append(model.embeddings)
      probs.append(prob)
      preds.append(pred)
      target.append(y)

  info = {
    'embeddings': torch.cat(embeddings),
    'prob': torch.cat(probs),
    'preds': torch.cat(preds),
    'target': torch.cat(target)
  }
  return tot_loss / len(dataset), n_correct / len(dataset), info

### <font color='gold'>**Funzione `train`**</font>

Definiamo la vera e propria funzione di training. La funzione implementa un algoritmo di [Early stopping](https://en.wikipedia.org/wiki/Early_stopping), tecnica di regolarizzazione che permette di trainare un modello per evitare l'overfitting. Questo algoritmo si basa sul concetto di *pazienza*, cioè il numero di epoche che possono trascorrere senza che la validation loss diminuisca prima che il training termini.

Detto in altri termini, l'early stopping funziona come segue:

1. considera un parametro `patience`, e una variabile `count` (inizialmente pari a 0)
2. al termine di ogni epoca, calcola la validation loss
3. se la validation loss è <font color='green'>minore</font> di quella all'epoca precedente, continua il training per l'epoca successiva
4. altirmenti, se la loss è <font color='red'>maggiore</font> dell'epoca aggiorna count come segue: `count = count + 1`
5. se `count == patience` (è stato raggiunto il livello di *pazienza*), allora termina il training

**Nota 1:** <font color="aqua">perché usare la pazienza, e non interrompere quando la validation loss scende?</font>
- <u>si ha overfitting</u> quando la training loss continua a scendere ma la validation loss inizia a salire
- tuttavia, può non essere una buona idea arrestare il training appena si incontra una validation loss minore rispetto all'epoca precedente
- infatti, capita spesso che in una qualche epoca successiva, la loss diminuisca nuovamente

**Nota 2:** <font color='aqua'>tenere traccia del checkpoint (stato dei pesi della rete) migliore</font>
- è buona norma mantenere traccia dei pesi della rete ogniqualvolta la validation loss migliora
- in questo modo, alla fine del training possiamo ottenere la rete con la <u>minima validation loss</u>

In [ ]:
def train(model, train_dataset, val_dataset, args):
  optimizer = args.optim_class(model.parameters(), lr=args.lr)
  dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=2)

  train_losses, val_losses = dict(), dict()
  step = -1
  count = 0
  best_val_loss = float('inf')
  best_model = None

  ### INIZIO TRAINING
  for epoch in range(args.n_epochs):

    tot_loss = 0
    num_examples = 0
    n_correct = 0

    loading_bar = tqdm(dataloader)
    model.train()

    ### EPOCA DI TRAINING
    for (X, y) in loading_bar:
      X, y = X.to(args.device), y.to(args.device)

      # probabilità classi
      prob = model(X)

      # media loss sul batch
      loss = args.loss_fn(prob, y)

      # azzeramento gradiente, backprop, e ottimizzazione
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      step += 1

      # log step
      tot_loss += loss.item() * X.shape[0]
      num_examples += X.shape[0]
      running_loss = tot_loss / num_examples
      train_losses[step] = running_loss

      pred = prob.argmax(-1)
      n_correct += sum(pred == y)

      loading_bar.set_description(f'Epoch {epoch+1:<3d} [Loss: {running_loss:.4f}]')
    ### FINE EPOCA DI TRANING

    train_accuracy = n_correct / len(train_dataset)
    val_loss, val_accuracy, _ = evaluate(model, val_dataset, args)
    val_losses[step] = val_loss

    print('-'*80)
    print(f'Train accuracy: {train_accuracy:.2%}')
    print(f'Val accuracy:   {val_accuracy:.2%}')
    print(f'Val loss:       {val_loss:.4f}')

    # early stopping
    if val_loss > best_val_loss:
      count += 1
      print(f'===> Patience {count:>3d}/{args.patience:<3d}')
      if count == args.patience:
        break
    else:
      count = 0
      best_val_loss = val_loss
      best_model = model.state_dict()

    print()
  ### FINE TRAINING

  model.load_state_dict(best_model)

  return train_losses, val_losses

### <font color='gold'>**Definizione degli iperparametri di training**</font>

Definiamo gli iperparametri di training. Usiamo la **Cross-Entropy Loss**, spesso migliore del *Mean-Squared Error* nei problemi di classificazione:

$$
H_p(q) = - \sum_{c=1}^Cq(y_c) \log p(y_c)
$$

Come algoritmo di ottimizzazione usiamo *Adam*, spesso migliore del classico *Stochastic Gradient Descent*.

Salviamo i pesi della rete per eseguire dei confronti con quelli al termine del training.

In [ ]:
args = SimpleNamespace(
    loss_fn = nn.CrossEntropyLoss(),
    optim_class = torch.optim.Adam,
    batch_size = 1024,
    lr = 0.0005,
    n_epochs = 20,
    patience = 20,
    device = 'cuda' if torch.cuda.is_available() else 'cpu',
    seed = 42
)

torch.manual_seed(args.seed)
model = CNN().to(args.device)

torch.save(model.state_dict(), 'model_before_train.pt')

### <font color='gold'>**Training!**</font> 🧠

Siamo pronti per trainare la rete. <u>**Nota:**</u> su **Colab con runtime T4 GPU** il training impiega circa <font color='red'>2</font> minuti.

Salviamo i pesi del modello dopo il training e training e validation loss.

In [ ]:
train_losses, val_losses = train(model, train_dataset, val_dataset, args)

torch.save({'train': train_losses, 'val': val_losses}, 'losses.pt')
torch.save(model.state_dict(), 'model_after_train.pt')

### 💡<font color='lightblue'>**Approfondimento: visualizzazione delle rappresentazioni apprese dalla rete**</font>

Possiamo visualizzare come la rete ha imparato a rappresentare al suo interno ciascun esempio.

Nel definire la funzione `forward` della rete, abbiamo programmato il salvataggio della rappresentazione interna della rete prima del livello di output, come mostrato qui:

In [ ]:
print(model.__repr__().replace('(out)', '\033[96m>>>>>>>>>>>>>>> SALVIAMO QUI GLI EMBEDDINGS (INPUT DEL LIVELLO DI OUTPUT) <<<<<<<<<<<<<<<\033[0m\n  (out)'))

Visualizziamo con la PCA le rappresentazioni vettoriali di ciascun esempio prese prima della porzione della rete chiamata `out` (che ci restituisce le probabilità di appartenenza ad una classe).

In [ ]:
model.load_state_dict(torch.load('model_before_train.pt'))
_, _, info = evaluate(model, train_dataset, args, no_loading_bar=False)
plot_dataset_visualization(info['embeddings'].cpu(), info['target'].cpu(), PCA(2), 'Rappresentazione interna prima del training')
plt.show()
print('\n')

model.load_state_dict(torch.load('model_after_train.pt'))
_, _, info = evaluate(model, train_dataset, args, no_loading_bar=False)
plot_dataset_visualization(info['embeddings'].cpu(), info['target'].cpu(), PCA(2), 'Rappresentazione interna $\mathbf{dopo}$ il training')

### <font color='gold'>**Visualizzazione training loss e validation loss**</font>

Ora possiamo visualizzare il grafico delle training loss e validation loss. Sovrapponiamo le due curve in modo da vedere come progrediscono durante il trainig. Visualizziamo anche le curve in scala logarimica (sul numero di *step*)

In [ ]:
losses_dict = torch.load('losses.pt')
train_losses, val_losses = losses_dict['train'], losses_dict['val']

plt.subplots(1,2, figsize=(16,4))
for i, (func, title) in enumerate([(plt.plot, 'Loss'), (plt.semilogx, 'Loss (log scale)')]):
  plt.subplot(1,2,i+1)
  func(list(train_losses.keys()), list(train_losses.values()), label='Train loss')
  func(list(val_losses.keys()), list(val_losses.values()), label='Val loss')
  idx = np.argmin(list(val_losses.values()))
  plt.axvline(list(val_losses.keys())[idx], color='red', label='Best val loss', linestyle='dashed')
  plt.legend()
  plt.title(title)
  plt.subplots_adjust(wspace=0.1)
  plt.xlabel('step')
  plt.grid('on', 'both')

### <font color='gold'>**Testing del modello**</font>

Terminato il training, possiamo testare le performance del nostro modello. In primis, calcoliamo l'accuratezza delle predizioni.

In [ ]:
_, test_accuracy, info = evaluate(model, test_dataset, args)
print(f'Test accuracy: {test_accuracy:.4f}')

Ora visualizziamo la matrice di confusione, in modo da osservare gli errori più comuni effettuati dal modello.

In [ ]:
target, preds = info['target'].cpu(), info['preds'].cpu()
ConfusionMatrixDisplay.from_predictions(target, preds)
plt.gca().set_yticklabels(target_names)
plt.gca().set_xticklabels(target_names)
plt.xticks(rotation = 45)
plt.title('Confusion matrix')
plt.show()

### 💡<font color='lightblue'>**Approfondimento: classificazione scorretta di *Shirts***</font>

Si può osservare che gli esempi della classe *Shirt* sono spesso classificati erroneamente.

In [ ]:
print('Percentuale di esempi con classificazione errata\n' + '-'*100)
for y in range(len(target_names)):
  misclassified_y = (preds != target) & (target == y)
  all_y = target == y

  print(f'{target_names[y]:>12}: {sum(misclassified_y)/sum(all_y):.2%}')

Se siamo curiosi, possiamo andare a vedere le *Shirt* classificate in maniera errata, e possiamo ispezionare le probabilità di ciascuna classe.

In [ ]:
# filtiramo le "shirt" del test set che non sono state classificate correttamente
misclassified_shirts = (preds != target) & (target == 6)
indexes = misclassified_shirts.nonzero().flatten()
probs = info['prob'].cpu()[misclassified_shirts]

# griglia 4 x 3
rows, cols = 4, 3
fig, _ = plt.subplots(rows, 2 * cols, figsize=(12,10))
fig.suptitle(f'Predizioni del modello (true class: {target_names[6]})', y=0.9)

for i in range(0, rows * cols * 2, 2):
    plt.subplot(rows, 2 * cols, i+1)
    x, y = test_dataset[indexes[i]]
    plt.imshow(1-x[0], cmap='gray')
    plt.axis('off')

    plt.subplot(rows, 2 * cols, i+2)
    # stampiamo un diagramma a torta delle probabilità superiori allo 0.01%
    probs_i = np.round(100 * probs[i].numpy(), decimals=2)
    patches, _ = plt.pie(probs_i)
    patches = [patches[i] for i, p in enumerate(probs_i) if p >= 0.01]
    labels = [f'{target_names[i]} - {p:.2f}%' for i, p in enumerate(probs_i) if p >= 0.01]
    plt.legend(patches, labels, loc='upper center', fontsize=8, framealpha=0.5)

plt.subplots_adjust(wspace=0.01, hspace=0.01)

## Conclusione

Risolvere un problema di Machine Learning richiede quasi sempre *trial and error*, soprattutto nella scelta degli iperparametri.

I curiosi possono provare a migliorare le performance del modello. <font color='gold'><u>**Cosa modificare?**</u></font>

1) Aiutandosi con la [documentazione di Pytorch](https://pytorch.org/docs/stable/nn.html), provare a variare nel modello:
* il numero di layer convoluzionali
* parametri dei layer convoluzionali e di pooling
  * kernel size
  * padding
  * stride
* il numero di strati *fully connected* (`Linear`) e, in ognuno, il numero di neuroni
* la presenza di strati `Dropout` (eventualmente modificando il *dropout rate*)
* la presenza di strati `BatchNorm2d`

2) Variare gli iperparametri di apprendimento:
* il numero massimo di epoche
* il numero di epoche in cui "pazientare"
* la dimensione dei mini-batch
* l'algoritmo di apprendimento (es. `SGD`)
* learning rate

**Con questo dataset e usando una rete convoluzionale relativamente semplice, l'accuracy potrà superare senza problemi il 90%.**